---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [123]:
import networkx as nx
#%matplotlib notebook
# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [124]:
def answer_one():
    # create network
    G = nx.read_edgelist("email_network.txt", data=[('time', float)], create_using=nx.MultiDiGraph())
    
    # check that all nodes are string type, only return G id this is True
    check_node_str = [True if False not in [True for n in G.nodes() if type(n) == str] else False]
    if check_node_str[0]:
        return G

In [125]:
answer_one()

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [126]:
def answer_two():
        
    G = answer_one()
    
    # employees are nodes, and emails are edges
    return len(G.nodes(data=True)), len(G.edges(data=True))

In [127]:
answer_two()

(167, 82927)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [128]:
def answer_three():
        
    # part1 check for strong connectivity
    # part2 check for weak connectivity

    G = answer_one()
    
    return nx.is_strongly_connected(G), nx.is_weakly_connected(G)

In [129]:
answer_three()

(False, True)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [130]:
def answer_four():
        
    G = answer_one()
    # create list of sets of connected nodes
    weakly_connected_components = nx.weakly_connected_components(G)
    # create a list of the length of these connected nodes
    lengths_of_sets = [len(x) for x in weakly_connected_components]
    
    # returns the max set of nodes
    return max(lengths_of_sets)

In [131]:
answer_four()

167

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [132]:
def answer_five():
        
    G = answer_one()
    # create list of sets of connected nodes
    strongly_connected_components = nx.strongly_connected_components(G)
    # create a list of the length of these connected nodes
    lengths_of_sets = [len(x) for x in strongly_connected_components]
    
    # returns the max set of nodes
    return max(lengths_of_sets)

In [133]:
answer_five()

126

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [134]:
def answer_six():

    G = answer_one()
    # creates a network of strongly connected nodes
    G_sc = max(nx.strongly_connected_component_subgraphs(G), key=len)
    
    return G_sc

In [135]:
answer_six()

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [136]:
def answer_seven():
        
    G_sc = answer_six()
    av_dis = nx.average_shortest_path_length(G_sc)
    
    return av_dis

In [137]:
answer_seven()

1.6461587301587302

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [138]:
def answer_eight():
        
    G_sc = answer_six()
    
    return max(nx.eccentricity(G_sc).values())

In [139]:
answer_eight()

3

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [140]:
def answer_nine():
       
    G_sc = answer_six()
    d = nx.diameter(G_sc)
    ecc = nx.eccentricity(G_sc)

    return set([k for k,v in ecc.items() if v == d])

In [141]:
answer_nine()

{'129', '134', '97'}

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [142]:
def answer_ten():
        
    G_sc = answer_six()
    r = nx.radius(G_sc)
    ecc = nx.eccentricity(G_sc)

    return set([k for k,v in ecc.items() if v == r])

In [143]:
answer_ten()

{'38'}

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [144]:
def answer_eleven():
        
    G_sc = answer_six()
    d = nx.diameter(G_sc)

    # creates a dictionary of dictionaries, each dictionary is one nodes links to all other nodes
    # using the shortest path method
    path = nx.shortest_path(G_sc)

    # create a list of all the shortest paths equal to the diameter for all nodes
    short_path = []
    for k in path.keys():
        for n,l in path[k].items():
            if (len(l)-1) == d:
                short_path.append(l)

    # count the number of times a starting node shows up
    counts = []
    for x in short_path:
        counts.append(x[0])

    # returns a tuple of the node with most connections and the number of connections
    return (max(counts), counts.count(max(counts)))

In [145]:
answer_eleven()

('97', 63)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [146]:
def answer_twelve():
        
    G_sc = answer_six()
    center = nx.center(G_sc)
    
    return len(nx.minimum_node_cut(G_sc, center[0], answer_eleven()[0]))

In [147]:
answer_twelve()

5

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [148]:
def answer_thirteen():
        
    G_sc = answer_six()
    G_un = nx.Graph(G_sc)
    
    return G_un

In [149]:
answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [150]:
def answer_fourteen():
        
    G_un = answer_thirteen()

    return (nx.transitivity(G_un), nx.average_clustering(G_un))

In [151]:
answer_fourteen()

(0.570111160700385, 0.697527243723142)